# Thomas Solver Overlay Test

This notebook compares the FPGA implementation of a tridiagonal matrix solver against a NumPy implementation. It measures the execution time of each approach and checks that both produce the same result.

In [10]:
from pynq import Overlay, allocate
import numpy as np
import time

np.set_printoptions(precision=4, suppress=True)


In [11]:
# Load overlay and inspect available IP blocks
overlay = Overlay('custom_tomas_solver_v1.bit')
# Replace 'thomas_solver_0' with the actual IP name from the printed dictionary if different
solver_ip = overlay.thomas_solver_0

In [12]:
N = 64
# Constants describing the tridiagonal matrix
dp = np.complex64(4+0j)
dp1 = np.complex64(3+0j)
dp2 = np.complex64(3+0j)
off = np.complex64(1+0j)

# Allocate buffers accessible to the FPGA
a_b = allocate(shape=(N,), dtype=np.complex64)
a_x = allocate(shape=(N,), dtype=np.complex64)

# Random right-hand side vector
b_np = (np.random.rand(N) + 1j*np.random.rand(N)).astype(np.complex64)
a_b[:] = b_np
a_x.fill(0)
a_b.flush()
a_x.flush()
print('Input vector b', b_np)


Input vector b [0.9141+0.2686j 0.4551+0.7976j 0.3513+0.2684j 0.284 +0.9619j
 0.7815+0.4927j 0.2275+0.891j  0.3806+0.9172j 0.0495+0.478j
 0.2552+0.0953j 0.7201+0.5495j 0.5861+0.8005j 0.4889+0.601j
 0.5873+0.3728j 0.8947+0.129j  0.4116+0.8828j 0.6522+0.7316j
 0.5816+0.3477j 0.3404+0.9465j 0.2085+0.4867j 0.9248+0.0877j
 0.7295+0.5121j 0.6251+0.3814j 0.4602+0.4612j 0.953 +0.3741j
 0.0928+0.8212j 0.2334+0.091j  0.2059+0.6664j 0.4291+0.929j
 0.8403+0.6256j 0.4249+0.894j  0.9489+0.6242j 0.6765+0.03j
 0.6492+0.1133j 0.8278+0.9202j 0.7796+0.7289j 0.1505+0.3567j
 0.5946+0.6956j 0.0142+0.6297j 0.9578+0.4876j 0.3588+0.6312j
 0.9942+0.6034j 0.5767+0.6552j 0.0265+0.1522j 0.4656+0.4203j
 0.1881+0.6939j 0.5865+0.0406j 0.4214+0.1869j 0.6582+0.1274j
 0.1308+0.8068j 0.5059+0.229j  0.6821+0.9419j 0.0847+0.3653j
 0.4368+0.4226j 0.4857+0.9976j 0.73  +0.2084j 0.9065+0.9736j
 0.1799+0.6274j 0.5232+0.7405j 0.5347+0.591j  0.343 +0.7847j
 0.3168+0.2156j 0.2964+0.1947j 0.3557+0.018j  0.8219+0.1297j]


In [13]:
def thomas_solver_numpy(dp, dp1, dp2, off, b):
    N = b.shape[0]
    c_prime = np.empty(N, dtype=np.complex64)
    d_prime = np.empty(N, dtype=np.complex64)
    inv = 1.0/np.complex64(dp1)
    c_prime[0] = off * inv
    d_prime[0] = b[0] * inv
    for i in range(1, N-1):
        denom = dp - off * c_prime[i-1]
        inv = 1.0/denom
        c_prime[i] = off * inv
        d_prime[i] = (b[i] - off * d_prime[i-1]) * inv
    denom = dp2 - off * c_prime[N-2]
    d_prime[N-1] = (b[N-1] - off * d_prime[N-2]) / denom
    x = np.empty(N, dtype=np.complex64)
    x[-1] = d_prime[-1]
    for i in range(N-2, -1, -1):
        x[i] = d_prime[i] - c_prime[i] * x[i+1]
    return x

In [14]:
# CPU reference implementation
t0 = time.time()
x_ref = thomas_solver_numpy(dp, dp1, dp2, off, b_np)
cpu_time = time.time() - t0
print(f'CPU time: {cpu_time*1e3:.3f} ms')


CPU time: 5.525 ms


In [15]:
a_rm = solver_ip.register_map
print(a_rm)

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  dp_1 = Register(dp=write-only),
  dp_2 = Register(dp=write-only),
  dp1_1 = Register(dp1=write-only),
  dp1_2 = Register(dp1=write-only),
  dp2_1 = Register(dp2=write-only),
  dp2_2 = Register(dp2=write-only),
  off_1 = Register(off=write-only),
  off_2 = Register(off=write-only)
}


In [16]:
# Configure solver IP
rm = solver_ip.register_map
rm.dp_r = float(np.real(dp))
rm.dp_i = float(np.imag(dp))
rm.dp1_r = float(np.real(dp1))
rm.dp1_i = float(np.imag(dp1))
rm.dp2_r = float(np.real(dp2))
rm.dp2_i = float(np.imag(dp2))
rm.off_r = float(np.real(off))
rm.off_i = float(np.imag(off))
rm.b = a_b.physical_address
rm.x = a_x.physical_address

# Run hardware solver
t0 = time.time()
rm.CTRL.AP_START = 1
while True:
    rm = solver_ip.register_map
    if rm.CTRL.AP_DONE:
        break
hw_time = time.time() - t0
print(f'Hardware time: {hw_time*1e3:.3f} ms')
if hw_time > 0:
    print(f'Speedup (CPU/HW): {cpu_time/hw_time:.2f}x')


Hardware time: 0.701 ms
Speedup (CPU/HW): 7.88x


In [17]:
# Compare results
a_x.invalidate()
x_hw = np.array(a_x)
print('Results match:', np.allclose(x_ref, x_hw, atol=1e-6))
print('CPU result x_ref', x_ref)
print('Hardware result x_hw', x_hw)


Results match: False
CPU result x_ref [ 0.2986+0.021j   0.0183+0.2057j  0.0832-0.0463j  0.0002+0.2479j
  0.2002+0.0168j -0.0196+0.1776j  0.1055+0.1638j -0.022 +0.0845j
  0.0318-0.0237j  0.1499+0.1057j  0.0888+0.1505j  0.0808+0.0928j
  0.0767+0.0791j  0.1997-0.0366j  0.0194+0.1963j  0.1343+0.1341j
  0.0954-0.0012j  0.0655+0.2184j -0.017 +0.0741j  0.2109-0.0279j
  0.0981+0.1254j  0.1263+0.0384j  0.0218+0.1024j  0.2466+0.0133j
 -0.0551+0.2186j  0.0667-0.0664j  0.0216+0.1379j  0.0529+0.1811j
  0.1959+0.0666j  0.0038+0.1779j  0.214 +0.1158j  0.0892-0.0169j
  0.1057-0.0182j  0.137 +0.2031j  0.174 +0.1259j -0.0535+0.0222j
  0.1904+0.142j  -0.1134+0.1056j  0.2773+0.0654j -0.0379+0.1204j
  0.2333+0.0841j  0.0988+0.1466j -0.0519-0.0152j  0.1351+0.0666j
 -0.0231+0.1692j  0.1455-0.0496j  0.0275+0.0697j  0.1658-0.0425j
 -0.0325+0.2276j  0.0952-0.061j   0.1578+0.2455j -0.0441+0.0211j
  0.1035+0.0356j  0.0671+0.2592j  0.114 -0.0747j  0.2071+0.2481j
 -0.036 +0.056j   0.1168+0.1553j  0.092 +0.0635j  0.

In [18]:
overlay.free()
a_b.free()
a_x.free()

AttributeError: 'PynqBuffer' object has no attribute 'free'